In [130]:
import urllib.request 
import urllib.parse
import json 

# naver api 
client_id =    "m77dd2kg43"
client_secret= "ShA5D0Dqgo6aidH6z2rjsdm1ijeKaKr1x8CYNNo6"

# locations = ["성남시 분당구", "용인시 수지구", "용인시 기흥구", "용인시 처인구",
#              "수원시 권선구", "수원시 영통구", "수원시 장안구","수원시 팔달구",
#              "안양시 만안구", "안양시 동안구", "경기 과천시","경기 광명시",
#             "경기도 광주시","경기 군포시","경기 의왕시","경기 이천시","경기 하남시"]

locations = ["성남시 분당구", "용인시",
             "수원시",
             "안양시", "경기 과천시",
            "경기도 광주시","경기 군포시","경기 의왕시","경기 하남시"]
headers = {
    "X-NCP-APIGW-API-KEY-ID": client_id,
    "X-NCP-APIGW-API-KEY": client_secret}

base_url = "https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode"
spots=[]
for location in locations:
    url = f"{base_url}?query={urllib.parse.quote(location)}"    # 요청 URL 생성

    # API 요청
    req = urllib.request.Request(url, headers=headers)
    try:
        with urllib.request.urlopen(req) as response:
            if response.status == 200:
                result = json.loads(response.read().decode("utf-8"))
                if result['addresses']:
                    latitude = result['addresses'][0]['y']  
                    longitude = result['addresses'][0]['x']  
                    print(f"{location}: 위도={latitude}, 경도={longitude}") 
                    spots.append({"location": location,
                                  "latitude": latitude,
                                  "longitude": longitude})
                else:
                    print(f"{location}: 좌표 못찾음요 ㅜ.")
            else:
                print(f"{location}: HTTP 에러 {response.status}")
    except Exception as e:
        print(f"{location}: 요청 실패 - {e}")



성남시 분당구: 위도=37.38282, 경도=127.118926
용인시: 위도=37.241252, 경도=127.177492
수원시: 위도=37.263476, 경도=127.028646
안양시: 위도=37.394291, 경도=126.956753
경기 과천시: 위도=37.429201, 경도=126.987675
경기 광명시: 위도=37.478618, 경도=126.86465
경기도 광주시: 위도=37.429431, 경도=127.255048
경기 군포시: 위도=37.361523, 경도=126.935338
경기 의왕시: 위도=37.344887, 경도=126.968279
경기 이천시: 위도=37.272342, 경도=127.435034
경기 하남시: 위도=37.539301, 경도=127.214874


In [134]:
import urllib.request
import json
import pandas as pd

load_url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"  # 운전 길찾기 API URL
start_location = next(spot for spot in spots if spot["location"] == "성남시 분당구") #밀집지역 !
destinations = [spot for spot in spots if spot["location"] != "성남시 분당구"] # rest of all
data = []
# 각 도착지에 대해 API 요청
for dest in destinations:
    url = f"{load_url}?start={start_location['longitude']},{start_location['latitude']}&goal={dest['longitude']},{dest['latitude']}"
    req = urllib.request.Request(url, headers=headers)
    try:
        with urllib.request.urlopen(req) as response:
            if response.status == 200:
                result = json.loads(response.read().decode("utf-8"))
                # Debugging: Print the entire response to check structure
                # print(f"Response for {dest['location']}: {json.dumps(result, indent=4)}")
                if "route" in result and "traoptimal" in result["route"]:
                    total_distance = 0
                    if "guide" in result["route"]["traoptimal"][0]:
                        # Summing the distances from each guide step
                        for guide in result["route"]["traoptimal"][0]["guide"]:
                            total_distance += guide["distance"]
                        data.append({
                            "Destination": dest["location"],
                            "Distance (meters)": total_distance
                        })    
                        print(f"Distance from {start_location['location']} to {dest['location']}: {total_distance} meters")
                    else:
                        print(f"{dest['location']}: 'guide' key missing in the response")
                else:
                    print(f"{dest['location']}: 'route' or 'traoptimal' key missing in the response")
            else:
                print(f"{dest['location']}: HTTP error {response.status}")
    except Exception as e:
        print(f"{dest['location']}: Request failed - {e}")



Distance from 성남시 분당구 to 용인시: 20644 meters
Distance from 성남시 분당구 to 수원시: 21796 meters
Distance from 성남시 분당구 to 안양시: 18932 meters
Distance from 성남시 분당구 to 경기 과천시: 19212 meters
Distance from 성남시 분당구 to 경기 광명시: 33012 meters
Distance from 성남시 분당구 to 경기도 광주시: 20965 meters
Distance from 성남시 분당구 to 경기 군포시: 22964 meters
Distance from 성남시 분당구 to 경기 의왕시: 20608 meters
Distance from 성남시 분당구 to 경기 이천시: 42957 meters
Distance from 성남시 분당구 to 경기 하남시: 26186 meters


In [78]:
youngin = (9224+13467+25164)//3
suwon = (30898+19746+21157+20676)//4
anyang = (21228+19375)//2 
print (youngin, suwon, anyang)

15951 23119 20301


In [101]:
import pandas as pd

# DataFrame을 만듭니다.
df = pd.DataFrame(data)

# 피벗 테이블 생성
distance_table = pd.pivot_table(df, values='Distance (meters)', columns='Destination', aggfunc='sum')

# 새로운 그룹 컬럼을 생성하는 함수
def group_destination(destination):
    if destination.startswith("용인시"):
        return "용인시"
    elif destination.startswith("수원시"):
        return "수원시"
    elif destination.startswith("안양시"):
        return "안양시"
    else:
        return destination  # 나머지 목적지는 그대로 둡니다.

# 'Destination'에 따라 새로운 그룹을 생성
grouped_columns = [group_destination(col) for col in distance_table.columns]

# 피벗 테이블의 컬럼을 새로 만든 그룹에 맞게 설정
distance_table.columns = grouped_columns

# 그룹별 평균을 계산하여 새로운 테이블을 생성
grouped_distance_table = distance_table.groupby(level=0, axis=1).mean().astype(int)

# 결과 출력
grouped_distance_table


C:\Users\hhm91\AppData\Local\Temp\ipykernel_34488\3447324348.py:27: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped_distance_table = distance_table.groupby(level=0, axis=1).mean().astype(int)


,경기 과천시,경기 광명시,경기 군포시,경기 의왕시,경기 이천시,경기 하남시,경기도 광주시,수원시,안양시,용인시
Distance (meters),20874,33012,22964,20642,42957,26487,20965,23122,20347,15938


In [135]:
import folium

with open('./data/json/map.json', encoding='utf-8') as file:
    korea_map = json.load(file)
# start_location = next(spot for spot in spots if spot["location"] == "성남시 분당구")
m = folium.Map(location=[float(start_location['latitude']),
                               float(start_location['longitude'])], zoom_start=10)
folium.Marker(
    location = [float(start_location['latitude']),float(start_location['longitude'])],
    popup = 'seoungnam',
    icon = folium.Icon(color="red")
).add_to(m)

for dest in destinations:
    folium.Marker(
    location = [float(dest['latitude']),float(dest['longitude'])],
    popup = 'neighbor',
    icon = folium.Icon(color="blue")
).add_to(m)

folium.CircleMarker(
    radius=150,
    location = [float(start_location['latitude']),float(start_location['longitude'])],
    popup = 'nearby',
    color = "crimson",
    fill = True
).add_to(m)
m